Víctor Rincón Yepes
Github: https://github.com/rinvictor/Algoritmos-Optimizacion-IA/blob/main/Algoritmos_R3.ipynb

In [20]:
import os
import urllib.request
import tsplib95
import random     

In [21]:
file = "swiss42.tsp"
file_gz = file + '.gz'

# Verificar si el archivo existe antes de descargarlo
if not os.path.exists(file):
    # Descargar el archivo comprimido si no existe
    urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file_gz)
    # Descomprimir el archivo
    os.system(f"gzip -d {file_gz}")
else:
    print(f"El archivo {file} ya existe en el sistema.")

El archivo swiss42.tsp ya existe en el sistema.


In [22]:
tsp = tsplib95.load(file)
nodos = list(tsp.get_nodes())
aristas = list(tsp.get_edges())

In [23]:
print(f"Total nodos: {len(nodos)}")
print(f"Total aristas: {len(aristas)}")

Total nodos: 42
Total aristas: 1764


### Implentación de un algoritmo de búsqueda aleatoria

Implementación de un algoritmo de búsqueda aleatoria para hacernos una idea de las dimensiones del problema.

In [24]:
# distancia total de un recorrido
def calcular_distancia_total(solucion, tsp):
  distancia_total = 0
  for i in range(len(solucion)-1):
    # problema.get_weight devuelve la distancia entre dos nodos
    distancia_total += tsp.get_weight(solucion[i] ,solucion[i+1]) 
  return distancia_total + tsp.get_weight(solucion[len(solucion)-1] ,solucion[0])

In [25]:
def generar_solucion(nodos):
    solucion_actual = [nodos[0]]
    for _ in nodos[1:]:
        nodos_restantes = set(nodos) - set({nodos[0]}) - set(solucion_actual)
        siguiente_nodo = random.choice(list(nodos_restantes))
        solucion_actual.append(siguiente_nodo)
    return solucion_actual

def busqueda_aleatoria(tsp, max_iteraciones=1000):
    nodos = list(tsp.get_nodes())
    
    mejor_solucion = []
    mejor_distancia = float('inf')

    for _ in range(max_iteraciones):
        solucion_actual = generar_solucion(nodos)
        distancia_actual = calcular_distancia_total(solucion_actual, tsp)

        if distancia_actual < mejor_distancia:
            mejor_solucion = solucion_actual
            mejor_distancia = distancia_actual
            
    return mejor_solucion, mejor_distancia

In [26]:
max_iteraciones = 10000 # Numero de iteraciones como condición de parada
mejor_solucion, mejor_distancia = busqueda_aleatoria(tsp, max_iteraciones=max_iteraciones)
print("La mejor solución encontrada es:", mejor_solucion)
print("Con una distancia total de:", mejor_distancia)

La mejor solución encontrada es: [0, 41, 22, 30, 28, 12, 14, 29, 20, 2, 33, 34, 17, 35, 13, 18, 39, 3, 11, 25, 5, 37, 26, 24, 38, 32, 9, 1, 27, 23, 40, 21, 8, 7, 31, 15, 36, 16, 6, 4, 10, 19]
Con una distancia total de: 3819


### Optimización por colonia de hormigas

El siguiente código implementa el algoritmo de colonia de hormigas para resolver el problema del viajero, donde las hormigas construyen soluciones parciales, se actualizan las feromonas en las aristas basadas en estas soluciones, y las feromonas se evaporan para permitir una exploración más amplia del espacio de búsqueda.

In [27]:
# Genera un nuevo nodo que la hormiga puede visitar
def generar_nodo(tsp, ruta_parcial_hormiga):
    nodos = list(tsp.get_nodes())
    return random.choice(list(set(range(1, len(nodos))) - set(ruta_parcial_hormiga)))

def incrementa_feromona(problem, feromonas, ruta_parcial_hormiga):
    for i in range(len(ruta_parcial_hormiga)-1):
        #  La cantidad de feromonas depositadas depende de la inversa de la distancia total de la ruta recorrida
        feromonas[ruta_parcial_hormiga[i]][ruta_parcial_hormiga[i+1]] += 1000 / calcular_distancia_total(ruta_parcial_hormiga, problem)
    return feromonas

# Reduce la cantidad de feromonas en base a una tasa de evaporación
def evaporar_feromonas(feromonas, evaporation_rate):
    for i in range(len(feromonas)):
        for j in range(len(feromonas[i])):
            feromonas[i][j] *= (1 - evaporation_rate)
    return feromonas

# Lógica del algoritmo de hormigas
def hormigas(tsp, max_iteraciones=1000, evaporation_rate=0.3):
    nodos = list(tsp.get_nodes())
    feromonas = [[1 for _ in range(len(nodos))] for _ in range(len(nodos))]

    mejor_solucion = []
    mejor_distancia = float('inf')

    for _ in range(max_iteraciones):
        Hormiga = [0]
        for _ in range(len(nodos)-1):
            nuevo_nodo = generar_nodo(tsp, Hormiga)
            Hormiga.append(nuevo_nodo)

        feromonas = incrementa_feromona(tsp, feromonas, Hormiga)
        feromonas = evaporar_feromonas(feromonas, evaporation_rate)

        distancia_actual = calcular_distancia_total(Hormiga, tsp)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[:]
            mejor_distancia = distancia_actual

    return mejor_solucion, mejor_distancia




In [28]:
max_iteraciones = 1000 # Numero de iteraciones como condición de parada
mejor_solucion, mejor_distancia = hormigas(tsp, max_iteraciones=max_iteraciones, evaporation_rate=0.3)
print("La mejor solución encontrada es:", mejor_solucion)
print("Con una distancia total de:", mejor_distancia)

La mejor solución encontrada es: [0, 3, 28, 16, 14, 11, 22, 21, 34, 9, 24, 23, 8, 12, 27, 1, 33, 31, 20, 36, 19, 13, 37, 39, 5, 38, 35, 17, 10, 18, 6, 26, 41, 29, 15, 7, 4, 25, 40, 30, 2, 32]
Con una distancia total de: 3840


### Mejora del algoritmo de colonia de hormigas

Mejora de la implementación de colonia de hormigas implementada anteriormente sobre el TSP mediante la elección de nodo que tenga en consideración una función de probabilidad que depende de las feromonas.

In [29]:
#  Calcula las probabilidades de seleccionar cada nodo vecino como próximo nodo a visitar por la hormiga, basándose en las feromonas y las distancias entre nodos
def calcular_probabilidades(tsp, feromonas, ruta_parcial_hormiga, alpha=1, beta=2):
    nodos_restantes = list(set(range(len(list(tsp.get_nodes())))) - set(ruta_parcial_hormiga))
    sumatoria_probabilidades = 0
    probabilidades = []

    for nodo in nodos_restantes:
        numerador = feromonas[ruta_parcial_hormiga[-1]][nodo] ** alpha * (1 / tsp.get_weight(ruta_parcial_hormiga[-1], nodo)) ** beta
        denominador = sum((feromonas[ruta_parcial_hormiga[-1]][vecino] ** alpha) * (1 / tsp.get_weight(ruta_parcial_hormiga[-1], vecino)) ** beta for vecino in nodos_restantes)
        probabilidad = numerador / denominador
        sumatoria_probabilidades += probabilidad
        probabilidades.append((nodo, probabilidad))

    probabilidades = [(nodo, probabilidad / sumatoria_probabilidades) for nodo, probabilidad in probabilidades]
    return probabilidades

# Elige un nodo vecino basándose en las probabilidades calculadas
def elegir_nodo_con_probabilidad(probabilidades):
    r = random.random()
    acumulado = 0

    for nodo, probabilidad in probabilidades:
        acumulado += probabilidad
        if acumulado >= r:
            return nodo

def hormigas_con_probabilidad(tsp, max_iteraciones, alpha=1, beta=2, evaporation_rate=0.3):
    nodos = list(tsp.get_nodes())
    feromonas = [[1 for _ in range(len(nodos))] for _ in range(len(nodos))]

    mejor_solucion = []
    mejor_distancia = float('inf')

    for _ in range(max_iteraciones):
        Hormiga = [0]

        while len(Hormiga) < len(nodos):
            probabilidades = calcular_probabilidades(tsp, feromonas, Hormiga, alpha, beta)
            nuevo_nodo = elegir_nodo_con_probabilidad(probabilidades)
            Hormiga.append(nuevo_nodo)

        feromonas = incrementa_feromona(tsp, feromonas, Hormiga)
        feromonas = evaporar_feromonas(feromonas, evaporation_rate)

        distancia_actual = calcular_distancia_total(Hormiga, tsp)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[:]
            mejor_distancia = distancia_actual

    return mejor_solucion, mejor_distancia




In [30]:
max_iteraciones = 1000 # Numero de iteraciones como condición de parada
mejor_solucion, mejor_distancia = hormigas_con_probabilidad(tsp, max_iteraciones, alpha=1, beta=2, evaporation_rate=0.1)
print("La mejor solución encontrada es:", mejor_solucion)
print("Con una distancia total de:", mejor_distancia)

La mejor solución encontrada es: [0, 1, 6, 5, 26, 18, 12, 11, 10, 25, 19, 13, 16, 14, 15, 37, 7, 17, 31, 35, 36, 34, 20, 33, 32, 30, 29, 8, 41, 23, 9, 39, 21, 40, 24, 38, 22, 28, 27, 2, 3, 4]
Con una distancia total de: 1544


Como vemos la mejora es significativa ya que encuentra una distancia mucho más corta usando esta segunda aproximación.